In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import seaborn as sns
from sklearn.manifold import TSNE

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# =============================================================================
# 1. DATA PREPARATION & LOADING
# =============================================================================

def create_face_embedding_model(input_shape=(224, 224, 3), embedding_dim=128):
    """Create base CNN for face feature extraction"""
    inputs = keras.Input(shape=input_shape)
    
    # Using a simple CNN - replace with ResNet50, VGGFace, etc. for better results
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(256, 3, activation='relu', padding='same')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    embeddings = layers.Dense(embedding_dim, activation=None, name='embeddings')(x)
    
    return Model(inputs, embeddings, name='face_embedding_model')

# =============================================================================
# 2. MODEL 1: BINARY CROSS-ENTROPY (CLASSIFICATION APPROACH)
# =============================================================================

def create_bce_model(num_classes, input_shape=(224, 224, 3), embedding_dim=128):
    """Face verification using classification with BCE loss"""
    base_model = create_face_embedding_model(input_shape, embedding_dim)
    
    # Add classification head
    embeddings = base_model.output
    embeddings = layers.BatchNormalization()(embeddings)
    predictions = layers.Dense(num_classes, activation='softmax', name='predictions')(embeddings)
    
    model = Model(base_model.input, predictions, name='bce_face_model')
    return model, base_model

def compile_bce_model(model, learning_rate=1e-3):
    """Compile BCE model"""
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# =============================================================================
# 3. MODEL 2: CONTRASTIVE LOSS (SIAMESE NETWORK)
# =============================================================================

def contrastive_loss(margin=1.0):
    """Contrastive loss function for siamese networks"""
    def loss(y_true, y_pred):
        # y_true: 1 if same person, 0 if different person
        # y_pred: distance between embeddings
        square_pred = tf.square(y_pred)
        margin_square = tf.square(tf.maximum(margin - y_pred, 0))
        return tf.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)
    return loss

def euclidean_distance(vectors):
    """Compute euclidean distance between two vectors"""
    x, y = vectors
    sum_square = tf.reduce_sum(tf.square(x - y), axis=1, keepdims=True)
    return tf.sqrt(tf.maximum(sum_square, tf.keras.backend.epsilon()))

def create_contrastive_model(input_shape=(224, 224, 3), embedding_dim=128):
    """Create siamese network for contrastive learning"""
    base_model = create_face_embedding_model(input_shape, embedding_dim)
    
    # Create siamese architecture
    input_a = keras.Input(shape=input_shape)
    input_b = keras.Input(shape=input_shape)
    
    # Get embeddings for both inputs
    embedding_a = base_model(input_a)
    embedding_b = base_model(input_b)
    
    # Normalize embeddings
    embedding_a = layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(embedding_a)
    embedding_b = layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(embedding_b)
    
    # Compute distance
    distance = layers.Lambda(euclidean_distance, name='distance')([embedding_a, embedding_b])
    
    model = Model([input_a, input_b], distance, name='contrastive_model')
    return model, base_model

def compile_contrastive_model(model, learning_rate=1e-3, margin=1.0):
    """Compile contrastive model"""
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=contrastive_loss(margin=margin),
        metrics=['mae']
    )
    return model

# =============================================================================
# 4. MODEL 3: TRIPLET LOSS
# =============================================================================

def triplet_loss(alpha=0.2):
    """Triplet loss function"""
    def loss(y_true, y_pred):
        # y_pred contains [anchor, positive, negative] embeddings
        anchor, positive, negative = y_pred[:, 0], y_pred[:, 1], y_pred[:, 2]
        
        # Calculate distances
        pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
        neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)
        
        # Triplet loss
        basic_loss = pos_dist - neg_dist + alpha
        loss = tf.reduce_mean(tf.maximum(basic_loss, 0.0))
        return loss
    return loss

def create_triplet_model(input_shape=(224, 224, 3), embedding_dim=128):
    """Create triplet network"""
    base_model = create_face_embedding_model(input_shape, embedding_dim)
    
    # Three inputs: anchor, positive, negative
    anchor_input = keras.Input(shape=input_shape, name='anchor')
    positive_input = keras.Input(shape=input_shape, name='positive')
    negative_input = keras.Input(shape=input_shape, name='negative')
    
    # Get embeddings
    anchor_embedding = base_model(anchor_input)
    positive_embedding = base_model(positive_input)
    negative_embedding = base_model(negative_input)
    
    # Normalize embeddings
    anchor_embedding = layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(anchor_embedding)
    positive_embedding = layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(positive_embedding)
    negative_embedding = layers.Lambda(lambda x: tf.nn.l2_normalize(x, axis=1))(negative_embedding)
    
    # Combine embeddings for loss calculation
    merged_output = layers.concatenate([anchor_embedding, positive_embedding, negative_embedding], axis=1)
    
    model = Model([anchor_input, positive_input, negative_input], merged_output, name='triplet_model')
    return model, base_model

def compile_triplet_model(model, learning_rate=1e-3, alpha=0.2):
    """Compile triplet model"""
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=triplet_loss(alpha=alpha)
    )
    return model

# =============================================================================
# 5. TRAINING UTILITIES
# =============================================================================

class DebugCallback(keras.callbacks.Callback):
    """Custom callback to monitor training progress"""
    def __init__(self, model_name):
        super().__init__()
        self.model_name = model_name
        self.losses = []
        
    def on_epoch_end(self, epoch, logs=None):
        self.losses.append(logs.get('loss', 0))
        if epoch % 10 == 0:
            print(f"{self.model_name} - Epoch {epoch}: Loss = {logs.get('loss', 0):.4f}")

def create_callbacks(model_name, patience=10):
    """Create training callbacks"""
    return [
        keras.callbacks.EarlyStopping(patience=patience, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5),
        DebugCallback(model_name)
    ]

# =============================================================================
# 6. EVALUATION & VISUALIZATION
# =============================================================================

def compute_embeddings(model, data):
    """Extract embeddings from trained model"""
    if hasattr(model, 'layers') and len(model.layers) > 1:
        # For BCE model, extract from embedding layer
        embedding_model = Model(model.input, model.get_layer('embeddings').output)
        return embedding_model.predict(data)
    else:
        # For contrastive/triplet models, use base model
        return model.predict(data)

def plot_training_history(histories, model_names):
    """Plot training histories for comparison"""
    plt.figure(figsize=(15, 5))
    
    # Plot losses
    plt.subplot(1, 3, 1)
    for i, (history, name) in enumerate(zip(histories, model_names)):
        plt.plot(history.history['loss'], label=f'{name} Train')
        if 'val_loss' in history.history:
            plt.plot(history.history['val_loss'], label=f'{name} Val', linestyle='--')
    plt.title('Training Loss Comparison')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    # Plot accuracy (for BCE model)
    plt.subplot(1, 3, 2)
    for i, (history, name) in enumerate(zip(histories, model_names)):
        if 'accuracy' in history.history:
            plt.plot(history.history['accuracy'], label=f'{name} Train')
            if 'val_accuracy' in history.history:
                plt.plot(history.history['val_accuracy'], label=f'{name} Val', linestyle='--')
    plt.title('Training Accuracy (BCE Model)')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    # Plot learning rate
    plt.subplot(1, 3, 3)
    for i, (history, name) in enumerate(zip(histories, model_names)):
        if 'lr' in history.history:
            plt.plot(history.history['lr'], label=name)
    plt.title('Learning Rate Schedule')
    plt.xlabel('Epoch')
    plt.ylabel('Learning Rate')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

def plot_embeddings_tsne(embeddings, labels, title="Embedding Visualization"):
    """Visualize embeddings using t-SNE"""
    tsne = TSNE(n_components=2, random_state=42)
    embeddings_2d = tsne.fit_transform(embeddings)
    
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1], c=labels, cmap='tab10', alpha=0.7)
    plt.title(title)
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.colorbar(scatter)
    plt.grid(True, alpha=0.3)
    plt.show()

def compute_verification_metrics(embeddings1, embeddings2, labels):
    """Compute face verification metrics"""
    # Calculate distances
    distances = np.linalg.norm(embeddings1 - embeddings2, axis=1)
    
    # Compute ROC curve
    fpr, tpr, thresholds = roc_curve(labels, -distances)  # Negative distances for similarity
    roc_auc = auc(fpr, tpr)
    
    # Find Equal Error Rate (EER)
    fnr = 1 - tpr
    eer_idx = np.nanargmin(np.absolute(fnr - fpr))
    eer = fpr[eer_idx]
    
    return fpr, tpr, roc_auc, eer, thresholds[eer_idx]

def plot_roc_comparison(results_dict):
    """Plot ROC curves for all models"""
    plt.figure(figsize=(10, 8))
    
    for model_name, (fpr, tpr, auc_score, eer) in results_dict.items():
        plt.plot(fpr, tpr, label=f'{model_name} (AUC = {auc_score:.3f}, EER = {eer:.3f})', linewidth=2)
    
    plt.plot([0, 1], [0, 1], 'k--', alpha=0.5, label='Random Classifier')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves Comparison: Face Verification Models')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)
    plt.show()

def plot_distance_distributions(genuine_distances, impostor_distances, model_name):
    """Plot distance distributions for genuine and impostor pairs"""
    plt.figure(figsize=(10, 6))
    
    plt.hist(genuine_distances, bins=50, alpha=0.7, label='Genuine Pairs', color='green', density=True)
    plt.hist(impostor_distances, bins=50, alpha=0.7, label='Impostor Pairs', color='red', density=True)
    
    plt.xlabel('Distance')
    plt.ylabel('Density')
    plt.title(f'Distance Distribution - {model_name}')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# =============================================================================
# 7. DEBUGGING UTILITIES
# =============================================================================

def debug_model_architecture(model, model_name):
    """Print model architecture for debugging"""
    print(f"\n=== {model_name} Architecture ===")
    model.summary()
    print(f"Total parameters: {model.count_params():,}")
    
    # Check if gradients flow
    print(f"Trainable weights: {len(model.trainable_weights)}")
    print(f"Non-trainable weights: {len(model.non_trainable_weights)}")

def debug_data_shapes(data_dict):
    """Debug data shapes and types"""
    print("\n=== Data Shape Debug ===")
    for name, data in data_dict.items():
        if isinstance(data, (list, tuple)):
            print(f"{name}: {len(data)} items, shapes: {[d.shape for d in data]}")
        else:
            print(f"{name}: {data.shape}, dtype: {data.dtype}")

def sanity_check_model(model, sample_data, model_name):
    """Perform sanity checks on model"""
    print(f"\n=== Sanity Check: {model_name} ===")
    
    # Forward pass test
    try:
        if isinstance(sample_data, (list, tuple)):
            output = model(sample_data)
        else:
            output = model([sample_data])
        print(f"✓ Forward pass successful, output shape: {output.shape}")
    except Exception as e:
        print(f"✗ Forward pass failed: {e}")
        return False
    
    # Gradient test
    try:
        with tf.GradientTape() as tape:
            if isinstance(sample_data, (list, tuple)):
                prediction = model(sample_data, training=True)
            else:
                prediction = model([sample_data], training=True)
            loss = tf.reduce_mean(prediction)
        
        gradients = tape.gradient(loss, model.trainable_variables)
        non_none_grads = [g for g in gradients if g is not None]
        print(f"✓ Gradient computation successful, {len(non_none_grads)}/{len(gradients)} variables have gradients")
    except Exception as e:
        print(f"✗ Gradient computation failed: {e}")
        return False
    
    return True

# =============================================================================
# 8. EXAMPLE USAGE & MAIN WORKFLOW
# =============================================================================

def main_comparison_workflow():
    """Example workflow for comparing the three models"""
    print("Face Verification Models Comparison Workflow")
    print("=" * 50)
    
    # Model parameters
    input_shape = (224, 224, 3)
    embedding_dim = 128
    num_classes = 1000  # Adjust based on your dataset
    
    # Create models
    print("Creating models...")
    bce_model, bce_base = create_bce_model(num_classes, input_shape, embedding_dim)
    contrastive_model, contrastive_base = create_contrastive_model(input_shape, embedding_dim)
    triplet_model, triplet_base = create_triplet_model(input_shape, embedding_dim)
    
    # Compile models
    compile_bce_model(bce_model)
    compile_contrastive_model(contrastive_model)
    compile_triplet_model(triplet_model)
    
    # Debug architectures
    debug_model_architecture(bce_model, "BCE Model")
    debug_model_architecture(contrastive_model, "Contrastive Model")
    debug_model_architecture(triplet_model, "Triplet Model")
    
    # Create sample data for sanity checks
    sample_image = np.random.random((1,) + input_shape).astype(np.float32)
    sample_pair = [sample_image, sample_image]
    sample_triplet = [sample_image, sample_image, sample_image]
    
    # Sanity checks
    sanity_check_model(bce_model, sample_image, "BCE Model")
    sanity_check_model(contrastive_model, sample_pair, "Contrastive Model")
    sanity_check_model(triplet_model, sample_triplet, "Triplet Model")
    
    print("\n✓ All models created and validated successfully!")
    print("Next steps:")
    print("1. Load your face dataset (LFW, VGGFace2, etc.)")
    print("2. Preprocess and create appropriate data loaders")
    print("3. Train each model with the provided compile functions")
    print("4. Use the evaluation functions to compare performance")
    
    return {
        'bce_model': bce_model,
        'contrastive_model': contrastive_model,
        'triplet_model': triplet_model,
        'bce_base': bce_base,
        'contrastive_base': contrastive_base,
        'triplet_base': triplet_base
    }

if __name__ == "__main__":
    models = main_comparison_workflow()

2025-07-22 17:12:55.227824: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-22 17:12:55.227869: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-22 17:12:55.229227: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-22 17:12:55.239993: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-22 17:12:58.842999: W tensorflow/compiler/tf2

Face Verification Models Comparison Workflow
Creating models...


2025-07-22 17:13:06.123743: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-07-22 17:13:06.268671: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...



=== BCE Model Architecture ===


Model: "bce_face_model"

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ anchor (InputLayer) │ (None, 224, 224,  │       0 │ -                    │
│                     │ 3)                │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ positive            │ (None, 224, 224,  │       0 │ -                    │
│ (InputLayer)        │ 3)                │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ negative            │ (None, 224, 224,  │       0 │ -                    │
│ (InputLayer)        │ 3)                │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ face_embedding_mod… │ (None, 128)       │ 421,312 │ anchor[0][0],        │
│ (Functional)        │                   │         │ positive[0][0],      │
│                     │                   │         │ negative[0][0]       │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ lambda_2 (Lambda)   │ (None, 128)       │       0 │ face_embedding_mode… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ lambda_3 (Lambda)   │ (None, 128)       │       0 │ face_embedding_mode… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ lambda_4 (Lambda)   │ (None, 128)       │       0 │ face_embedding_mode… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ concatenate         │ (None, 384)       │       0 │ lambda_2[0][0],      │
│ (Concatenate)       │                   │         │ lambda_3[0][0],      │
│                     │                   │         │ lambda_4[0][0]       │
└─────────────────────┴───────────────────┴─────────┴──────────────────────┘

 Total params: 421,312 (1.61 MB)

 Trainable params: 421,312 (1.61 MB)

 Non-trainable params: 0 (0.00 B)

Total parameters: 421,312
Trainable weights: 10
Non-trainable weights: 0

=== Sanity Check: BCE Model ===
✓ Forward pass successful, output shape: (1, 1000)
✓ Gradient computation successful, 14/14 variables have gradients

=== Sanity Check: Contrastive Model ===
✓ Forward pass successful, output shape: (1, 1)
✓ Gradient computation successful, 10/10 variables have gradients

=== Sanity Check: Triplet Model ===
✓ Forward pass successful, output shape: (1, 384)
✓ Gradient computation successful, 10/10 variables have gradients

✓ All models created and validated successfully!
Next steps:
1. Load your face dataset (LFW, VGGFace2, etc.)
2. Preprocess and create appropriate data loaders
3. Train each model with the provided compile functions
4. Use the evaluation functions to compare performance


In [ ]:
#!/usr/bin/env python3
"""
Complete Face Verification Models Comparison
Trains and compares BCE, Contrastive, and Triplet Loss models
"""

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Reduce TF warnings

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import time
import json

# Import our custom modules (assumes previous scripts are saved as modules)
# from face_verifier_models import *  # Your model definitions
# from face_data_pipeline import *     # Your data pipeline

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

class FaceVerificationComparison:
    """Main class to orchestrate the complete comparison"""
    
    def __init__(self, dataset_path, dataset_type='lfw', image_size=(224, 224)):
        self.dataset_path = dataset_path
        self.dataset_type = dataset_type
        self.image_size = image_size
        self.results = {}
        self.models = {}
        self.histories = {}
        
    def setup_experiment(self):
        """Setup the complete experiment"""
        print("🚀 Setting up Face Verification Comparison Experiment")
        print("=" * 60)
        
        # 1. Load and prepare data
        print("📊 Loading and preparing dataset...")
        self.data_pipeline = complete_training_workflow(
            self.dataset_path, self.dataset_type
        )
        
        # 2. Create models
        print("\n🏗️  Creating models...")
        self.create_models()
        
        # 3. Show experiment info
        self.show_experiment_info()
        
    def create_models(self):
        """Create all three models"""
        num_classes = self.data_pipeline['num_classes']
        
        # BCE Model
        bce_model, bce_base = create_bce_model(
            num_classes, self.image_size + (3,), embedding_dim=128
        )
        compile_bce_model(bce_model, learning_rate=1e-3)
        
        # Contrastive Model
        contrastive_model, contrastive_base = create_contrastive_model(
            self.image_size + (3,), embedding_dim=128
        )
        compile_contrastive_model(contrastive_model, learning_rate=1e-3, margin=1.0)
        
        # Triplet Model
        triplet_model, triplet_base = create_triplet_model(
            self.image_size + (3,), embedding_dim=128
        )
        compile_triplet_model(triplet_model, learning_rate=1e-3, alpha=0.2)
        
        self.models = {
            'BCE': {'full': bce_model, 'base': bce_base},
            'Contrastive': {'full': contrastive_model, 'base': contrastive_base},
            'Triplet': {'full': triplet_model, 'base': triplet_base}
        }
        
        print("✅ All models created successfully!")
        
    def show_experiment_info(self):
        """Display experiment information"""
        info = self.data_pipeline['dataset_info']
        
        print(f"\n📋 Experiment Configuration:")
        print(f"   Dataset: {self.dataset_type.upper()}")
        print(f"   Total Images: {info['total_images']:,}")
        print(f"   Identities: {info['num_identities']:,}")
        print(f"   Classes (BCE): {self.data_pipeline['num_classes']:,}")
        print(f"   Image Size: {self.image_size}")
        print(f"   Train/Val/Test Split: {info['splits']['train']}/{info['splits']['val']}/{info['splits']['test']}")
        
        print(f"\n🎯 Test Setup:")
        test_info = self.data_pipeline['test_data']
        print(f"   Verification Pairs: {len(test_info['pairs']):,}")
        print(f"   Positive Pairs: {sum(test_info['labels']):,}")
        print(f"   Negative Pairs: {len(test_info['labels']) - sum(test_info['labels']):,}")
        
    def train_all_models(self, epochs=30, quick_test=False):
        """Train all three models"""
        if quick_test:
            epochs = 3
            print("🏃 Quick test mode: Training for only 3 epochs")
        
        print(f"\n🎯 Training all models for {epochs} epochs...")
        
        generators = self.data_pipeline['generators']
        
        # Train BCE Model
        print("\n" + "="*50)
        print("📈 Training BCE (Classification) Model")
        print("="*50)
        start_time = time.time()
        
        self.histories['BCE'] = train_bce_model(
            self.models['BCE']['full'],
            generators['bce_train'],
            generators['bce_val'],
            epochs=epochs
        )
        
        bce_time = time.time() - start_time
        print(f"⏱️  BCE Training completed in {bce_time:.1f}s")
        
        # Train Contrastive Model
        print("\n" + "="*50)
        print("📈 Training Contrastive (Siamese) Model")
        print("="*50)
        start_time = time.time()
        
        self.histories['Contrastive'] = train_contrastive_model(
            self.models['Contrastive']['full'],
            generators['contrastive_train'],
            epochs=epochs
        )
        
        contrastive_time = time.time() - start_time
        print(f"⏱️  Contrastive Training completed in {contrastive_time:.1f}s")
        
        # Train Triplet Model
        print("\n" + "="*50)
        print("📈 Training Triplet Model")
        print("="*50)
        start_time = time.time()
        
        self.histories['Triplet'] = train_triplet_model(
            self.models['Triplet']['full'],
            generators['triplet_train'],
            epochs=epochs
        )
        
        triplet_time = time.time() - start_time
        print(f"⏱️  Triplet Training completed in {triplet_time:.1f}s")
        
        # Store training times
        self.training_times = {
            'BCE': bce_time,
            'Contrastive': contrastive_time,
            'Triplet': triplet_time
        }
        
        print(f"\n✅ All models trained successfully!")
        
    def evaluate_all_models(self):
        """Evaluate all models on verification task"""
        print("\n" + "="*50)
        print("📊 Evaluating All Models")
        print("="*50)
        
        test_pairs = self.data_pipeline['test_data']['pairs']
        test_labels = self.data_pipeline['test_data']['labels']
        
        for model_name in ['BCE', 'Contrastive', 'Triplet']:
            print(f"\n🔍 Evaluating {model_name} Model...")
            
            fpr, tpr, roc_auc, eer = evaluate_model(
                self.models[model_name]['base'],
                test_pairs,
                test_labels,
                model_name
            )
            
            self.results[model_name] = {
                'fpr': fpr,
                'tpr': tpr,
                'roc_auc': roc_auc,
                'eer': eer,
                'training_time': self.training_times[model_name]
            }
        
        print("\n✅ All models evaluated!")
        
    def create_comprehensive_report(self):
        """Create comprehensive comparison report with visualizations"""
        print("\n" + "="*50)
        print("📈 Creating Comprehensive Comparison Report")
        print("="*50)
        
        # Create figure with subplots
        fig = plt.figure(figsize=(20, 12))
        
        # 1. Training Loss Comparison
        plt.subplot(2, 3, 1)
        for model_name, history in self.histories.items():
            epochs = range(1, len(history.history['loss']) + 1)
            plt.plot(epochs, history.history['loss'], label=f'{model_name} Train Loss')
            if 'val_loss' in history.history:
                plt.plot(epochs, history.history['val_loss'], label=f'{model_name} Val Loss', linestyle='--')       
        plt.title('Training Loss Comparison')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()
        # 2. Training Accuracy (BCE Model)
        plt.subplot(2, 3, 2)
        if 'accuracy' in self.histories['BCE'].history: 
            epochs = range(1, len(self.histories['BCE'].history['accuracy']) + 1)
            plt.plot(epochs, self.histories['BCE'].history['accuracy'], label='BCE Train Accuracy')
            if 'val_accuracy' in self.histories['BCE'].history:
                plt.plot(epochs, self.histories['BCE'].history['val_accuracy'], label='BCE Val Accuracy', linestyle='--')
        plt.title('Training Accuracy (BCE Model)')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()      
        